In [ ]:
##############################
### Place all imports here ###
##############################
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
import os
import sys
from elpv_reader import load_dataset
from PIL import Image, ImageOps

# Method 1:

# Method 2:

In [ ]:
images, proba, types = load_dataset()

In [ ]:
tempimages = images[:1000]
tempproba = proba[:1000]
temptypes = types[:1000]

validation_images = images[1000:1500]
validation_proba = proba[1000:1500]
validation_types = types[1000:1500]

testing_images = images[1500:]
testing_proba = proba[1500:]
testing_types = types[1500:]

In [ ]:
print(images.shape)

In [ ]:
training_images = np.zeros((3000, 300, 300))
training_proba = np.zeros(3000)
training_types = np.chararray(3000, itemsize=4)

for i in range(1000):
    training_images[i] = tempimages[i]
    training_proba[i] = tempproba[i]
    training_types[i] = temptypes[i]
    
    image = Image.fromarray(tempimages[i])
    imflip = ImageOps.flip(image)
    flipped = np.array(imflip)
    training_images[i + 1000] = flipped
    training_proba[i + 1000] = tempproba[i]
    training_types[i + 1000] = temptypes[i]
    
    image2 = Image.fromarray(tempimages[i])
    imflip2 = ImageOps.mirror(image2)
    flipped2 = np.array(imflip2)
    training_images[i + 2000] = flipped2
    training_proba[i + 2000] = tempproba[i]
    training_types[i + 2000] = temptypes[i]

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((training_images, training_proba, training_types))
train_dataset = train_dataset.shuffle(3000).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((validation_images, validation_proba, validation_types))
val_dataset = val_dataset.shuffle(500).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((testing_images, testing_proba, testing_types))
test_dataset = test_dataset.shuffle(1000).batch(32)

In [ ]:
train_iterator = train_dataset.as_numpy_iterator()
train_batch = next(train_iterator)

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32, (3, 3), 1, activation='relu', input_shape=(300, 300, 1))) 
model.add(MaxPooling2D())

model.add(Conv2D(64, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(128, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
history = model.fit(train_dataset, epochs=10, callbacks=[tensorboard_callback])